<a href="https://colab.research.google.com/github/poseidon2022/Retreival-Augumented-Generation/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade fsspec==2024.6.1
!pip install -qU \
  langchain==0.0.300 \
  datasets==2.14.6 \
  pinecone-client==2.2.4 \
  tiktoken==0.5.1
!pip install langchain_google_genai
!pip install pypdf==3.1.0

In [55]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KE')
PINECONE_API = userdata.get('PIN_CONE')
PINECONE_ENV = 'gcp-starter'


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",
                             google_api_key = GOOGLE_API_KEY)

#Example generation
llm.invoke("Write me a ballad about LangChain").content

In [18]:
#let us have a continuous in-memory context remembering example
messages = [
    ("system", "You are a helpful assistant."),
    ("human", "Hi AI how are you today?"),
    ("ai", "I am great, how can I help you today?"),
    ("human", "I want to know more about string theory.")
]

In [ ]:
res = llm.invoke(messages)
res.content

In [ ]:
messages.append(("ai", res.content))
messages

In [ ]:
messages.append(("human", "tell me more"))
llm.invoke(messages).content

Now we have our llm setup and the next thing is loading up a freash document. I setup a document on a fictional company named Kidus Melaku Simegne. But you can load up any document and test the functionality.

In [102]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
import uuid
import time
from tqdm.auto import tqdm

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",
                                          google_api_key = GOOGLE_API_KEY)
#example embedding
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734449505806,
 0.01813092641532421]

In [ ]:
#Now we have to load up our document and try to split it into smalled chunks that
#are later to be used for embedding
def load_document(file_path):
  loader = PyPDFLoader(file_path)
  document = loader.load()

  #now we use a recursive text splitter to chunk up the text into a list of smaller texts
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
  )
  chunks = text_splitter.split_documents(document)

  #now let us extraxt the content from the chunks and use google's generative ai embedding
  #to prepare our embedding vector
  batch_size = 20
  ids = []
  context_array = []
  meta_data = []
  for i in tqdm(range(0, len(chunks), batch_size)):
    i_end = min(i + batch_size, len(chunks))
    batch = chunks[i:i_end]
    for idx, row in enumerate(batch):
      print(f"appending {idx + i}")
      ids.append(str(uuid.uuid4()))
      context_array.append(batch[idx].page_content)
      meta_data.append({
          'source' : row.metadata["source"],
          'page' : row.metadata["page"] + 1,
          'context' : batch[idx].page_content
      })


  emb_vectors = embeddings.embed_documents(context_array)
  return ids, emb_vectors, meta_data


#let us check if our embedding function is working correctly
ids, emb_vectors, meta_data = load_document("Kidus Melaku's company.pdf")
len(emb_vectors)

Now we have our embeding function and to insert any embeded vector along with its meta data and id to a database, we use a vector database known as pinecone for that. we will create an index with a specified dimension and we will try to upsert the embeded document to our database

In [138]:
import pinecone

#setting up the index
pinecone.init(api_key = PINECONE_API, environment = PINECONE_ENV)
index_list = pinecone.list_indexes() #this is to make sure that the same index is not
#created multiple times when running the application multiple times.

if not index_list:
  print("Creating index")
  pinecone.create_index("default", dimension = 768, metric = "cosine")

#One can change the metric to dotproduct and anu other, based on the application

#this is to make sure that our index is created with the dimensions defined and all
print(pinecone.describe_index("default"))
index = pinecone.Index("default")

Creating index
IndexDescription(name='default', metric='cosine', replicas=1, dimension=768.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')


In [139]:
#now let us create our upserting function for the embeded vector to pinecone

def upsert(ids, emb_vectors, meta_data, index):
   to_upsert = zip(ids, emb_vectors, meta_data)
   index.upsert(vectors = to_upsert)
   time.sleep(2)


In [ ]:
#let us tru upserting a sample document to the vector database
ids, emb_vectors, meta_data = load_document("Kidus Melaku's company.pdf")
upsert(ids, emb_vectors, meta_data, index)

In [142]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.00119,
 'namespaces': {'': {'vector_count': 119}},
 'total_vector_count': 119}

In [143]:
#now let us initialize our vector store from here
from langchain.vectorstores import Pinecone

text_field = "context"

vectorstore = Pinecone(index, embeddings.embed_query,text_field)

/usr/local/lib/python3.10/dist-packages/langchain/vectorstores/pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [147]:
#exampl match from our vector database
query = "what software technologies does the company offer"
vectorstore.similarity_search(query, k = 3)

[Document(page_content='ever-changing technology landscape. Whether it’s developing an IT strategy, optimizing existing \nsystems, or implementing new technologies, the company’s consultants have the knowledge and', metadata={'page': 3.0, 'source': "Kidus Melaku's company.pdf"}),
 Document(page_content='Simegne has the expertise and experience to deliver high-quality software that meets and \nexceeds client expectations. \nSub-services under Software Development :', metadata={'page': 2.0, 'source': "Kidus Melaku's company.pdf"}),
 Document(page_content='areas: \n1. Software Development \nAt the core of Kidus Melaku Simegne’s offerings is its software development service. The', metadata={'page': 2.0, 'source': "Kidus Melaku's company.pdf"})]

In [151]:
def augument_prompt(query:str):
  results = vectorstore.similarity_search(query, k=30)

  #the source knowledge to provide to our llm
  source_knowledge = "\n".join([x.page_content for x in results])

  augumented_prompt = f"""using the contexts below answer the query.

  Contexts: {source_knowledge}

  Query : {query}"""

  return augumented_prompt


print(augument_prompt(query))

using the contexts below answer the query.

  Contexts: ever-changing technology landscape. Whether it’s developing an IT strategy, optimizing existing 
systems, or implementing new technologies, the company’s consultants have the knowledge and
Simegne has the expertise and experience to deliver high-quality software that meets and 
exceeds client expectations. 
Sub-services under Software Development :
areas: 
1. Software Development 
At the core of Kidus Melaku Simegne’s offerings is its software development service. The
assurance services to ensure the reliability and performance of software products. 
2. Artificial Intelligence and Machine Learning
Sub-services under Software Development : 
 Custom Software Development : Designing and developing software solutions from 
scratch, tailored to the unique requirements of each client.
a passion for technology and a deep understanding of market needs. Initially, the company 
started as a small software development firm, focusing on cust